# Product inventory

In [29]:
class Product:

    def __init__(self, product_id, name, category, price = 2.5, quantity = 4):

        if not isinstance(quantity, int):
            raise TypeError("quantity must be an int")
        if not isinstance(price, (int, float)):
            raise TypeError("price must be a number")
        if quantity < 0:
            raise ValueError("quantity cannot be negative")
        if price < 0:
            raise ValueError("price cannot be negative")  
        
        self.product_id = product_id
        self.name = name
        self.category = category
        self.price = price
        self.quantity = quantity

    def value(self):
        return self.price * self.quantity
        print(f'Value: {self.price * self.quantity}')

    def adjust_qty(self, delta):

        if not isinstance(delta, int):
            raise TypeError("delta must be an int")
        new_qty = self.quantity + delta 
        if new_qty < 0:
            raise ValueError("Insufficient stock")
        self.quantity = new_qty
        print(f'Quantity updated to {self.quantity}')
        

In [30]:
class Inventory:

    def __init__(self):
        self._items = {}

    def add(self, product):         # ensure product is a Product; reject duplicate IDs
        if product.product_id in self._items:
            raise ValueError("Product id already exists.")
        self._items[product.product_id] = product

    def get(self, product_id):      # return Product or raise KeyError
        if product_id not in self._items:
            raise KeyError("No such product.")
        return self._items[product_id]

    def remove(self, product_id):   # delete by id or raise KeyError
        if product_id not in self._items:
            raise KeyError("No such product.")
            
        else:
            return self._items.pop(product_id)

    def restock(self, product_id, amount):
        if not isinstance(amount, int) or amount <= 0:
            raise ValueError("amount must be a positive integer")
        else:
            prod = self.get(product_id)               # raises KeyError if missing
            # this uses the method get which already return an error if such product does not exist
            return prod.adjust_qty(+amount)

    def sell(self, product_id, amount):
        if not isinstance(amount, int) or amount <= 0:
            raise ValueError("amount must be a positive integer")
        prod = self.get(product_id)
        
        if prod.quantity < amount:
            raise ValueError(f"Not enough stock for {prod.name}: have {prod.quantity}, need {amount}")
        return prod.adjust_qty(-amount)  

    def total_value(self):
        return sum(p.value() for p in self._items.values())

    def low_stock(self): 
        return [p for p in self._items.values() if p.quantity <= getattr(p, "min_qty", 0)]
            
    

In [31]:
p = Product("P1", "Pen", "Stationery", price = 2.5, quantity = 4)
print(p.name, p.quantity, p.value())   # expect: Pen 4 10.0

Pen 4 10.0


In [32]:
inv = Inventory()
p1 = Product("P1", "Pen", "Stationery", 2.5, 4)
p2 = Product("P2", "Notebook", "Stationery", 3.0, 1)

inv.add(p1)
inv.add(p2)

print(inv.get("P1").name)              # expect: Pen
print(round(inv.total_value(), 2))     # expect: 13.0  (2.5*4 + 3.0*1)


Pen
13.0


In [28]:
for pid, p in inv._items.items():
    print(pid, p.price, p.quantity, p.value())

P1 3 4 10.0
P2 3.0 1 10.0


In [12]:
inv = Inventory()
p = Product("P1", "Pen", "Stationery", 2.5, 4)
inv.add(p)

print(inv.get("P1").quantity)          # 4
inv.restock("P1", 3)
print(inv.get("P1").quantity)          # 7
inv.sell("P1", 5)
print(inv.get("P1").quantity)          # 2


4
Quantity updated to 7
7
Quantity updated to 2
2


In [13]:
inv = Inventory()
p = Product("P1", "Pen", "Stationery", 2.5, 1)
inv.add(p)

try:
    inv.sell("P1", 2)                  # would go negative
except Exception as e:
    print(type(e).__name__, e)         # expect: ValueError "insufficient stock" (or your message)


ValueError Not enough stock for Pen: have 1, need 2


In [14]:
inv = Inventory()
p = Product("P1", "Pen", "Stationery", 2.5, 4)
inv.add(p)

try:
    inv.add(p)
except Exception as e:
    print(type(e).__name__, e)         # expect: ValueError "Product id already exists."


ValueError Product id already exists.


In [15]:
inv = Inventory()
p1 = Product("P1", "Pen", "Stationery", 2.5, 4); p1.min_qty = 3
p2 = Product("P2", "Notebook", "Stationery", 3.0, 1); p2.min_qty = 2
p3 = Product("P3", "Tape", "Office", 1.0, 4)     # no min_qty -> treated as 0

inv.add(p1); inv.add(p2); inv.add(p3)

lows = inv.low_stock()
print([p.product_id for p in lows])     # expect: ['P2']  (since 1 <= 2)


['P2']


In [16]:
inv = Inventory()
p = Product("P1", "Pen", "Stationery", 2.5, 4)
inv.add(p)

removed = inv.remove("P1")
print(removed is p)                     # expect: True

try:
    inv.get("P1")
except Exception as e:
    print(type(e).__name__, e)          # expect: KeyError "No such product."


True
KeyError 'No such product.'


In [33]:
inv = Inventory()
p = Product("P1", "Pen", "Stationery", 2.5, 4)
inv.add(p)

for bad in [0, -3, 2.5, "5"]:
    try:
        inv.restock("P1", bad)
    except Exception as e:
        print("restock", bad, "->", type(e).__name__)  # expect: ValueError for <=0 or type error per your checks

for bad in [0, -1]:
    try:
        inv.sell("P1", bad)
    except Exception as e:
        print("sell", bad, "->", type(e).__name__)     # expect: ValueError for non-positive


restock 0 -> ValueError
restock -3 -> ValueError
restock 2.5 -> ValueError
restock 5 -> ValueError
sell 0 -> ValueError
sell -1 -> ValueError
